## Building a Text Retrieval Database

### Step 0: Create a connection with your database

In [5]:
import getpass
# This collects a masked password from the user
mypasswd = getpass.getpass()

········


In [6]:
mysso = 'djkgg'
dbname = 'dsa_student'
schema = 'djkgg'

In [8]:
import psycopg2
import numpy as np
from psycopg2.extensions import adapt, register_adapter, AsIs

# Then connects to the DB
connection = psycopg2.connect(database = dbname, 
                              user = mysso, 
                              host = 'pgsql.dsa.lan',
                              password = mypasswd)

### Step 1: Create data repository (i.e table) within a database.

```SQL

-------------------------
-- Basic Table 
-------------------------

CREATE TABLE sebcq5.BookSearch(
	id SERIAL NOT NULL,
	name varchar(250) NOT NULL,
	content text NOT NULL
);

ALTER TABLE sebcq5.BookSearch
ADD CONSTRAINT pk_BookSearch PRIMARY KEY (id);
```

In [9]:
CREATE_TABLES = f"""
DROP TABLE IF EXISTS {schema}.BookSearch;
CREATE TABLE {schema}.booksearch(
    id SERIAL NOT NULL,
    name varchar(250) NOT NULL,
    content text NOT NULL
);

ALTER TABLE {schema}.booksearch
ADD CONSTRAINT pk_BookSearch PRIMARY KEY (id);
"""
# print(CREATE_TABLES)

In [10]:
with connection, connection.cursor() as cursor:
    cursor.execute(CREATE_TABLES)

**Check whether the table is created by listing all the tables in the schema**

In [11]:
SQL_QUERY = f"""
SELECT table_schema, table_name
FROM information_schema.tables
WHERE table_schema = '{schema}';
"""

In [12]:
import pandas as pd

df = pd.read_sql_query(SQL_QUERY, connection)
df

table_schema  table_name
0        djkgg     f24dmir
1        djkgg   documents
2        djkgg  booksearch
3        djkgg   booklines

### Step 2: Add a column that implements the vector model, then parse the data into it.

```SQL
-------------------------
-- Separate Ts_Vector column
-------------------------
-- TS_Vector for GIN INDEX
ALTER TABLE sebcq5.BookSearch 
  ADD COLUMN content_tsv_gin tsvector;
  
UPDATE sebcq5.BookSearch 
SET content_tsv_gin = to_tsvector('pg_catalog.english', content);
```

In [13]:
UPDATE_TABLE=f"""
-------------------------
-- Separate Ts_Vector column
-------------------------
-- TS_Vector for GIN INDEX
ALTER TABLE {schema}.booksearch 
  ADD COLUMN content_tsv_gin tsvector;
  
UPDATE {schema}.booksearch 
SET content_tsv_gin = to_tsvector('pg_catalog.english', content);
"""
# print(UPDATE_TABLE)

In [14]:
with connection, connection.cursor() as cursor:
    cursor.execute(UPDATE_TABLE)

**Check the table**

In [15]:
SQL_QUERY = f"""
SELECT table_schema, table_name, column_name, data_type
FROM information_schema.columns
WHERE table_schema = '{schema}' AND table_name = 'booksearch';
"""

df = pd.read_sql_query(SQL_QUERY, connection)
df

table_schema  table_name      column_name          data_type
0        djkgg  booksearch               id            integer
1        djkgg  booksearch             name  character varying
2        djkgg  booksearch          content               text
3        djkgg  booksearch  content_tsv_gin           tsvector

### Step 3: Add another column that implements the vector model, then parse the data into it.

```SQL
-- TS_Vector for GIST INDEX
ALTER TABLE sebcq5.BookSearch 
  ADD COLUMN content_tsv_gist tsvector;

UPDATE sebcq5.BookSearch 
SET content_tsv_gist = to_tsvector('pg_catalog.english', content);
```

In [16]:
UPDATE_TABLE=f"""
-- TS_Vector for GIST INDEX
ALTER TABLE {schema}.booksearch 
  ADD COLUMN content_tsv_gist tsvector;

UPDATE {schema}.booksearch 
SET content_tsv_gist = to_tsvector('pg_catalog.english', content);
"""
# print(UPDATE_TABLE)

In [17]:
with connection, connection.cursor() as cursor:
    cursor.execute(UPDATE_TABLE)

**Check the table**

In [18]:
SQL_QUERY = f"""
SELECT table_schema, table_name, column_name, data_type
FROM information_schema.columns
WHERE table_schema = '{schema}' AND table_name = 'booksearch';
"""

df = pd.read_sql_query(SQL_QUERY, connection)
df

table_schema  table_name       column_name          data_type
0        djkgg  booksearch                id            integer
1        djkgg  booksearch              name  character varying
2        djkgg  booksearch           content               text
3        djkgg  booksearch   content_tsv_gin           tsvector
4        djkgg  booksearch  content_tsv_gist           tsvector

### Step 4: Set up database triggers to parse all new content loaded into the vector models.

```SQL
--TRIGGER
CREATE TRIGGER tsv_gin_update 
	BEFORE INSERT OR UPDATE
	ON sebcq5.BookSearch 
	FOR EACH ROW 
	EXECUTE PROCEDURE 
	tsvector_update_trigger(content_tsv_gin,'pg_catalog.english',content);

CREATE TRIGGER tsv_gist_update 
	BEFORE INSERT OR UPDATE
	ON sebcq5.BookSearch 
	FOR EACH ROW 
    EXECUTE PROCEDURE
	tsvector_update_trigger(content_tsv_gist,'pg_catalog.english',content);

```

In [19]:
CREATE_TRIGGER=f"""
--TRIGGER
CREATE TRIGGER tsv_gin_update 
    BEFORE INSERT OR UPDATE
    ON {schema}.booksearch 
    FOR EACH ROW 
    EXECUTE PROCEDURE 
    tsvector_update_trigger(content_tsv_gin,'pg_catalog.english',content);

CREATE TRIGGER tsv_gist_update 
    BEFORE INSERT OR UPDATE
    ON {schema}.booksearch 
    FOR EACH ROW 
    EXECUTE PROCEDURE
    tsvector_update_trigger(content_tsv_gist,'pg_catalog.english',content);
"""

In [20]:
with connection, connection.cursor() as cursor:
    cursor.execute(CREATE_TRIGGER)

### Step 5:  Add a specialized indexing to the vector models.

```SQL
-------------------------
-- Create Indexes
-------------------------

-- Index on content (Trigram needed,to use Gin Index)
-- CREATE EXTENSION pg_trgm;  -- Done by DB Admin

CREATE INDEX BookSearch_content
ON sebcq5.BookSearch USING GIN(content gin_trgm_ops);

-- GIN INDEX on content_tsv_gin
CREATE INDEX BookSearch_content_tsv_gin
ON sebcq5.BookSearch USING GIN(content_tsv_gin);

-- GIST INDEX on content_tsv_gist
CREATE INDEX BookSearch_content_tsv_gist
ON sebcq5.BookSearch USING GIST(content_tsv_gist);


```

In [21]:
CREATE_INDEX=f"""
-------------------------
-- Create Indexes
-------------------------

-- Index on content (Trigram needed,to use Gin Index)
-- CREATE EXTENSION pg_trgm;  -- Done by DB Admin

CREATE INDEX booksearch_content
ON {schema}.booksearch USING GIN(content gin_trgm_ops);

-- GIN INDEX on content_tsv_gin
CREATE INDEX booksearch_content_tsv_gin
ON {schema}.booksearch USING GIN(content_tsv_gin);

-- GIST INDEX on content_tsv_gist
CREATE INDEX booksearch_content_tsv_gist
ON {schema}.booksearch USING GIST(content_tsv_gist);
"""

In [22]:
with connection, connection.cursor() as cursor:
    cursor.execute(CREATE_INDEX)

### Step 6: Grant permission to read your table
```SQL
GRANT USAGE ON SCHEMA <yourpawprint> TO dsa_ro_user;
GRANT SELECT ON <yourpawprint>.BookSearch TO dsa_ro_user;
```

In [23]:
GRANT_ACCESS=f"""
GRANT USAGE ON SCHEMA {schema} TO dsa_ro_user;
GRANT SELECT ON {schema}.booksearch TO dsa_ro_user;
"""

In [24]:
with connection, connection.cursor() as cursor:
    cursor.execute(GRANT_ACCESS)